### Content
    0. File Path
    1. Load Vertices
    2. Load Edges
    3. Create Graph & Test
    
    (Following part will be added later)
    1. Add more attributes to Nodes and Edges
    -Nodes:
        Entity Node: mention
        Realation Node: Sentence/Clause/Entity
        Type Node: add type node to Mention Part by Hearst pattern 
        
        -Weight: similarity score with token
            Entity Node:
            Type Node: 
            Relation Node: do not calculate is better because Relation Nodes like "Sentence" contains more info.
            
    -Edges:
        Type edges should be added
        
        -Weight: all edges weight is "1"
        
    2. Compute GST


In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
sc.addPyFile('/usr/local/Cellar/apache-spark/2.4.1/libexec/jars/graphframes-0.7.0-spark2.4-s_2.11.jar')

In [4]:
#1. RDD -> DataFrame -> GraphFrame (code need to be change when add attr)
#2. GraphX -> GraphFrame (code no need to change when add attr) encounter problem：https://github.com/graphframes/graphframes/issues/359

In [5]:
import pyspark.sql.functions as f

In [6]:
from graphframes import *

## 0. File Path

In [7]:
File_verticesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Vertices/part-*"

In [8]:
File_edgesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Edges/part-*"

## 1. Load Vertices

In [9]:
verticesText0 = spark.read.csv(File_verticesTextRDD, header='false', inferSchema='true', sep='\t')

In [10]:
verticesText0.show(10)
# Document：(id, DocumentProperty(title, url, domain, tweets))
# Sentence：(id, SentenceProperty(sentenceNumber, sentenceContent, root, links.toList(lenth!=0)))
# Clause：  (id, ClauseProperty(clause,clauseType,subject,relation,argument,root,additional.toList))
# Mention： (id, MentionProperty(mention, NERtype, entity))
# Entity：  (id, EntityProperty(entity))

+--------+---+---+--------------------+-------------------+------+
|     _c0|_c1|_c2|                 _c3|                _c4|   _c5|
+--------+---+---+--------------------+-------------------+------+
|Sentence|  0|  1|Politics of Burun...|Politics of Burundi| $END$|
|Sentence|  1|  2|Executive power i...|Politics of Burundi| $END$|
|Sentence|  2|  3|Legislative power...|Politics of Burundi| $END$|
|Sentence|  3|  4|The political lan...|Politics of Burundi| $END$|
|Sentence|  4|  5|The current Presi...|Politics of Burundi| $END$|
|Sentence|  5|  6|Nkurunziza was th...|Politics of Burundi| $END$|
|Sentence|  6|  7|In November 1995,...|Politics of Burundi| $END$|
|Sentence|  7|  8|In July 1996, for...|Politics of Burundi| $END$|
|Sentence|  8|  9|He declared himse...|Politics of Burundi| $END$|
|Sentence|  9| 10|Widespread condem...|Politics of Burundi| $END$|
+--------+---+---+--------------------+-------------------+------+
only showing top 10 rows



In [ ]:
##exmaple:
# Document	5691614	Office of the United Nations High Commissioner for Human Rights			List() $END$
# Document	5691615	Gecko			List() $END$
# Document	5691616	Maeshowe			List() $END$
# Document	5691617	Kiwi			List() $END$
# Document	5691618	Iblis			List() $END$
# Document	5691619	Number			List() $END$

# Sentence	0	1	Politics of Burundi takes place in a framework of a transitional presidential representative democratic republic, whereby the President of Burundi is both head of state and head of government, and of a multi-party system.	Politics of Burundi	 $END$
# Sentence	1	2	Executive power is exercised by the government.	Politics of Burundi	 $END$

# Clause	24971	("This method", "was used", "by an estimated 38 million couples worldwide", "in 1991")	SVA	This method	was used	by an estimated 38 million couples worldwide	Coitus interruptus-3	in 1991		 $END$
# Clause	24972	("what", "was transpiring", "quickly", "spread and everybody")	UNKNOWN	what	was transpiring	quickly	First Council of Constantinople-36	spread and everybody		 $END$
# Clause	24973	("The news of what was transpiring quickly spread and everybody", "rushed", "to the church")	SVA	The news of what was transpiring quickly spread and everybody	rushed	to the church	First Council of Constantinople-36		 $END$

# Mention	24975	Freedom House	MISC	Freedom_House $END$
# Mention	24976	NGO	ORGANIZATION	Non-governmental_organization $END$
# Mention	24977	Obiang	PERSON	Teodoro_Obiang_Nguema_Mbasogo $END$

# Entity	8593089	Josephine_of_Leuchtenberg $END$
# Entity	8593090	Public_Auditorium $END$
# Entity	8593091	West_Plains,_Missouri $END$

In [11]:
verticesText1 = verticesText0.select("_c0","_c1","_c2","_c3","_c4")\
.withColumnRenamed("_c0", "name").withColumnRenamed("_c1", "id")\
.withColumnRenamed("_c2", "attr1").withColumnRenamed("_c3", "attr2")\
.withColumnRenamed("_c4", "attr3")

In [12]:
verticesText1.show(10)

+--------+---+-----+--------------------+-------------------+
|    name| id|attr1|               attr2|              attr3|
+--------+---+-----+--------------------+-------------------+
|Sentence|  0|    1|Politics of Burun...|Politics of Burundi|
|Sentence|  1|    2|Executive power i...|Politics of Burundi|
|Sentence|  2|    3|Legislative power...|Politics of Burundi|
|Sentence|  3|    4|The political lan...|Politics of Burundi|
|Sentence|  4|    5|The current Presi...|Politics of Burundi|
|Sentence|  5|    6|Nkurunziza was th...|Politics of Burundi|
|Sentence|  6|    7|In November 1995,...|Politics of Burundi|
|Sentence|  7|    8|In July 1996, for...|Politics of Burundi|
|Sentence|  8|    9|He declared himse...|Politics of Burundi|
|Sentence|  9|   10|Widespread condem...|Politics of Burundi|
+--------+---+-----+--------------------+-------------------+
only showing top 10 rows



## 2. Load Edges

In [13]:
edgesText0 = spark.read.csv(File_edgesTextRDD)

In [14]:
edgesText0.show(10)

+------------+-------+--------------------+
|         _c0|    _c1|                 _c2|
+------------+-------+--------------------+
|Edge(5691617|7284099|contains the sent...|
|Edge(5691617|7284100|contains the sent...|
|Edge(5691617|7284101|contains the sent...|
|Edge(5691617|7284102|contains the sent...|
|Edge(5691617|7284103|contains the sent...|
|Edge(5691617|7284104|contains the sent...|
|Edge(5691617|7284105|contains the sent...|
|Edge(5691617|7284106|contains the sent...|
|Edge(5691617|7284107|contains the sent...|
|Edge(5691617|7284108|contains the sent...|
+------------+-------+--------------------+
only showing top 10 rows



In [15]:
edgesText0=edgesText0.select(f.translate(f.col("_c0"), "Edge(", "").alias("src"), "_c1", f.translate(f.col("_c2"), ")", "").alias("label"))

In [16]:
edgesText0.show()

+-------+-------+--------------------+
|    src|    _c1|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [17]:
edgesText1 = edgesText0.select("*").withColumnRenamed("_c1", "dst")

In [18]:
edgesText1.show()

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [41]:
verticesText1.filter("name = 'Clause'").show()

+------+----+--------------------+-----+--------------------+
|  name|  id|               attr1|attr2|               attr3|
+------+----+--------------------+-----+--------------------+
|Clause|4559|("A significant c...|  SVA|A significant con...|
|Clause|4560|("by the French r...|  SVO|by the French res...|
|Clause|4561|("quinine and str...|  SVC|quinine and stryc...|
|Clause|4562|("strychnine", "i...|  SVC|          strychnine|
|Clause|4563|("Some writers", ...|  SVO|        Some writers|
|Clause|4564|("those", "would ...|  SVO|               those|
|Clause|4565|("The English Cha...|  SVC|The English Chann...|
|Clause|4566|("Johnston", "res...|  SVO|            Johnston|
|Clause|4567|("Tupper", "becam...|  SVO|              Tupper|
|Clause|4568|("His father", "m...|  SVA|          His father|
|Clause|4569|("Many physicists...|  SVA|     Many physicists|
|Clause|4570|("of quantum mech...|  SVC|of quantum mechanics|
|Clause|4571|("The King", "app...|  SVO|            The King|
|Clause|

## 3. Create Graph & Test

In [19]:
Graph = GraphFrame(verticesText1, edgesText1)

In [20]:
Graph.edges.show(10)

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
+-------+-------+--------------------+
only showing top 10 rows



In [23]:
Graph.vertices.show(10)

+--------+---+-----+--------------------+-------------------+
|    name| id|attr1|               attr2|              attr3|
+--------+---+-----+--------------------+-------------------+
|Sentence|  0|    1|Politics of Burun...|Politics of Burundi|
|Sentence|  1|    2|Executive power i...|Politics of Burundi|
|Sentence|  2|    3|Legislative power...|Politics of Burundi|
|Sentence|  3|    4|The political lan...|Politics of Burundi|
|Sentence|  4|    5|The current Presi...|Politics of Burundi|
|Sentence|  5|    6|Nkurunziza was th...|Politics of Burundi|
|Sentence|  6|    7|In November 1995,...|Politics of Burundi|
|Sentence|  7|    8|In July 1996, for...|Politics of Burundi|
|Sentence|  8|    9|He declared himse...|Politics of Burundi|
|Sentence|  9|   10|Widespread condem...|Politics of Burundi|
+--------+---+-----+--------------------+-------------------+
only showing top 10 rows



In [21]:
Graph.vertices.count()

8593092

In [22]:
Graph.edges.count()

11607488

In [32]:
# Example 1: bfs Algorithm
filteredPaths = Graph.bfs(
  fromExpr = "id = '5691617'",
  toExpr = "id = '7284108'",
  maxPathLength = 2)

In [33]:
# display(filteredPaths)
filteredPaths.show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[Document, 569161...|[5691617, 7284108...|[Sentence, 728410...|
+--------------------+--------------------+--------------------+

